In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os

# Define the directory containing the CSV files
directory = r'C:\Users\sarav\OneDrive\Desktop\SIH DATAS\onion datasets'

# User-defined future prediction end date
future_end_date = '2013-08-31'

# Iterate over all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Load the dataset
        file_path = os.path.join(directory, filename)
        zone_data = pd.read_csv(file_path)

        # Handle missing values (e.g., forward fill)
        zone_data.fillna(method='ffill', inplace=True)

        # Convert 'date' to datetime and set it as the index
        zone_data['date'] = pd.to_datetime(zone_data['date'])
        zone_data.set_index('date', inplace=True)

        # Normalize the 'price' column
        scaler = MinMaxScaler(feature_range=(0, 1))
        zone_data['price'] = scaler.fit_transform(zone_data[['price']])

        # Create training and testing datasets
        def create_dataset(dataset, time_step=1):
            X, Y = [], []
            for i in range(len(dataset) - time_step):
                X.append(dataset[i:(i + time_step)])
                Y.append(dataset[i + time_step])
            return np.array(X), np.array(Y)

        # Use 70% of the data for training and 30% for testing
        train_size = int(len(zone_data) * 0.7)
        test_size = len(zone_data) - train_size
        train_data, test_data = zone_data['price'].values[:train_size], zone_data['price'].values[train_size:]

        # Reshape the data
        time_step = 20  # Increased time step
        X_train, Y_train = create_dataset(train_data, time_step)
        X_test, Y_test = create_dataset(test_data, time_step)

        # Reshape input to be [samples, time steps, features] as required by LSTM
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

        # Build the LSTM model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(25))
        model.add(Dense(1))

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with validation split
        model.fit(X_train, Y_train, batch_size=1, epochs=100, validation_split=0.2, callbacks=[early_stop])

        # Make predictions
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform to get actual prices
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        Y_train = scaler.inverse_transform(Y_train.reshape(-1, 1))
        Y_test = scaler.inverse_transform(Y_test.reshape(-1, 1))

        # Adjust the index length to match the predictions
        train_index = zone_data.index[time_step:train_size]
        test_index = zone_data.index[train_size + time_step:]

        # Visualize the results
        plt.figure(figsize=(14,8))
        plt.plot(zone_data.index, scaler.inverse_transform(zone_data[['price']]), label='Original Price')

        plt.plot(train_index, train_predict, label='Train Prediction', color='orange')
        plt.plot(test_index, test_predict, label='Test Prediction', color='green')
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'{filename} - Price Prediction using LSTM')
        plt.show()

        # Future Price Prediction (up to user-specified date)
        future_dates = pd.date_range(start=zone_data.index[-1], end=future_end_date, freq='D')
        future_prices = []

        last_values = test_data[-time_step:]
        last_values = last_values.reshape((1, time_step, 1))

        for _ in range(len(future_dates)):
            predicted_price = model.predict(last_values)
            future_prices.append(predicted_price[0][0])
            last_values = np.append(last_values[:, 1:, :], predicted_price.reshape(1, 1, 1), axis=1)

        # Inverse transform to get actual price values
        future_prices = scaler.inverse_transform(np.array(future_prices).reshape(-1, 1))

        # Append future prices to the existing data for plotting
        future_df = pd.DataFrame(data=future_prices, index=future_dates, columns=['Future Price'])

        # Plot the predicted future prices
        plt.figure(figsize=(14,8))
        plt.plot(zone_data.index, scaler.inverse_transform(zone_data[['price']]), label='Original Price')
        plt.plot(future_df.index, future_df['Future Price'], label='Predicted Future Price', color='red')
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'{filename} - Future Price Prediction using LSTM')
        plt.show()

        # Buffer Stock Optimization (Basic Example)
        def buffer_stock_suggestion(predicted_prices):
            # Suggest more stock if prices are expected to drop
            stock_suggestion = []
            for i in range(1, len(predicted_prices)):
                if predicted_prices[i] < predicted_prices[i-1]:
                    stock_suggestion.append('Increase Stock')
                else:
                    stock_suggestion.append('Decrease Stock')
            return stock_suggestion

        # Generate buffer stock suggestions for future prices
        buffer_stock_suggestions = buffer_stock_suggestion(future_prices)

        # Display the future prices and buffer stock suggestions
        print(f"Future Prices for {filename}:")
        print(future_prices)
        print(f"Buffer Stock Suggestions for {filename}:")
        print(buffer_stock_suggestions)
